In [45]:
#!pip install demoji
#!python -m spacy download fr_core_news_md

In [2]:
import numpy
import pandas as pd
import demoji
import nltk
from nltk.corpus import stopwords
import re
import spacy

In [3]:
nlp = spacy.load('fr_core_news_md')

In [16]:
### Importation et préparation de la base de données
tweets=pd.read_csv("Jeux de données/tweets.csv", index_col=0)

Certains tweets peuvent avoir été postés à différentes heures par le même auteur ou peuvent être des retweets. Nous supprimons ces tweets.

In [32]:
tweets.drop_duplicates(subset = "Tweet", keep = 'first', inplace=True, ignore_index = True)
tweets.head(10)

,User,Tweet,timeTweet,source,retweet
0,Arki987,@pabliigoo @SNCF Celui de 14h50,2022-12-16 13:50:02+00:00,Twitter for iPhone,0
1,Im_Gi_,@Etohzz @SNCF @Elysee *,2022-12-16 13:49:50+00:00,Twitter for iPhone,0
2,Rayn_TN,@SNCF tgv inoui 2730 a destination de paris,2022-12-16 13:49:20+00:00,Twitter for iPhone,0
3,pabliigoo,@Arki987 @SNCF wesh comment ça,2022-12-16 13:48:30+00:00,Twitter for Android,0
4,dirtyswift,@ov_ostan @SNCF Mais bro… La ils me mettent en...,2022-12-16 13:47:53+00:00,Twitter for iPhone,0
5,Arki987,Train suivant supprimé 👌\r\n\r\nJe vous hais @...,2022-12-16 13:46:36+00:00,Twitter for iPhone,0
6,Rayn_TN,@SNCF le tgv bloquer à reims depuis 30min qui ...,2022-12-16 13:46:33+00:00,Twitter for iPhone,0
7,jabarre81,@SNCF @GroupeSNCF Bonjour merci je viens de vo...,2022-12-16 13:46:08+00:00,Twitter for Android,0
8,saint_dolly,Ce serait bien de mettre l'application à jour ...,2022-12-16 13:44:10+00:00,Twitter Web App,0
9,Rayn_TN,@SNCF c’est quand mon tgv il part svp il se pa...,2022-12-16 13:42:50+00:00,Twitter for iPhone,0


## Nettoyage des tweets

In [39]:
def transforme(texte):
    texte = texte.lower() # mettre les mots en minuscule
    #retirer les liens
    for item in re.compile("http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+").findall(texte):
        texte=texte.replace(item, "")
    # enlever le retour à la ligne
    texte = texte.replace("\n", " ").replace("\r", "") 
    # supprimer ",", "!", "?", "%", "(",")","/",'"', "$","£", "_", "-", "+", "*", "µ", ":","&,"§" 
    texte = re.sub(r"[,\!\?\%\(\)\/\"\$\£\+\*\µ,\:\&\§]", " ", texte) 
    # retirer les hashtags #
    for item in re.compile("([#]\w+)").findall(texte):
        texte=texte.replace(item, "")
    # retirer les apostrophes '
    for item in re.compile("([\’])").findall(texte):
        texte=texte.replace(item, " ")
    for item in re.compile("([\'])").findall(texte):
        texte=texte.replace(item, " ")
    # retirer les points de suspension
    for item in re.compile("([.]{1,5})").findall(texte):
        texte=texte.replace(item, "") 
    # retirer les personnes tagées
    for item in re.compile("([@]\w+)").findall(texte):
        texte=texte.replace(item, "")
    # retirer les adresses mail
    for item in re.findall('\S+@\S+', texte) :
        texte=texte.replace(item, "")
    # retire les mots contenant des chiffres
    texte = re.sub(r"[A-Za-z\.]*[0-9]+[A-Za-z%°\.]*", "", texte)
    # retirer les emojis
    for item in demoji.findall(texte):
        texte=texte.replace(item,"")
    return texte

In [40]:
tweets["texte_propre"] = tweets["Tweet"].apply(lambda x: transforme(x))
tweets.head()

,User,Tweet,timeTweet,source,retweet,texte_propre
0,Arki987,@pabliigoo @SNCF Celui de 14h50,2022-12-16 13:50:02+00:00,Twitter for iPhone,0,celui de
1,Im_Gi_,@Etohzz @SNCF @Elysee *,2022-12-16 13:49:50+00:00,Twitter for iPhone,0,
2,Rayn_TN,@SNCF tgv inoui 2730 a destination de paris,2022-12-16 13:49:20+00:00,Twitter for iPhone,0,tgv inoui a destination de paris
3,pabliigoo,@Arki987 @SNCF wesh comment ça,2022-12-16 13:48:30+00:00,Twitter for Android,0,wesh comment ça
4,dirtyswift,@ov_ostan @SNCF Mais bro… La ils me mettent en...,2022-12-16 13:47:53+00:00,Twitter for iPhone,0,mais bro… la ils me mettent en galère…


In [47]:
def decoupe_lemmatise(text):
    nlp_text = nlp(text)
    dict_lemm = {token : token.lemma_ for token in nlp_text}
    # list_mot = text.split()
    list_mots = []
    for x in nlp_text : 
        if len(x) >= 0 :
            list_mots.append(dict_lemm[x])
    return ' '.join(list_mots)

In [49]:
tweets["texte_lemmatise"] = tweets["texte_propre"].apply(decoupe_lemmatise)
tweets.head()

,User,Tweet,timeTweet,source,retweet,texte_propre,texte_lemmatise
0,Arki987,@pabliigoo @SNCF Celui de 14h50,2022-12-16 13:50:02+00:00,Twitter for iPhone,0,celui de,celui de
1,Im_Gi_,@Etohzz @SNCF @Elysee *,2022-12-16 13:49:50+00:00,Twitter for iPhone,0,,
2,Rayn_TN,@SNCF tgv inoui 2730 a destination de paris,2022-12-16 13:49:20+00:00,Twitter for iPhone,0,tgv inoui a destination de paris,tgv inoui avoir destination de pari
3,pabliigoo,@Arki987 @SNCF wesh comment ça,2022-12-16 13:48:30+00:00,Twitter for Android,0,wesh comment ça,wesh comment cela
4,dirtyswift,@ov_ostan @SNCF Mais bro… La ils me mettent en...,2022-12-16 13:47:53+00:00,Twitter for iPhone,0,mais bro… la ils me mettent en galère…,mais bro … le il me mettre en galère …


In [51]:
#Liste des stopwords
stop_words = nlp.Defaults.stop_words
stop_words_contexte = ["train", "trains", "sncf", "ter", "après", "alors", "sans", "tgv", "réseau", "gare", "voie", "lors", "suite",
"conducteur", "voyageur", "voyageurs", "circulation", "agent", "agents","passager", "passagers", "entre", "plusieurs", "entre",
"circulations", "passage", "non", "avant", "voies", "circule", "plus"]
stop_words = list(stop_words) + stop_words_contexte
# stop_words 